# 트리의 앙상블 

- 정형 데이터
    - 표 형태
    - 대부분은 숫자를 가르킨다
    - 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘은 `앙상블 학습`(이 알고리즘은 대부분 결정 트리 기반으로 만들어져있음, 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘)

- 비정형 데이터
    - 글, 사진, 디지털 음악 등
    - 25년 기준으로 신경망 알고리즘보다는 트랜스포머 사용

## 랜덤 포레스트
- 대표적인 결정 트리 기반의 앙상블 학습 방법
- 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징
- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만들고 각 결정 트리의 예측을 사용해 최종 예측을 만듦
- 한 샘플이 중복되어 추출될 수도 있음
- 노드 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음
- 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련
    - 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻게 되어 과대적합을 줄이고 일반화 성능을 높이는데 도움이 됨

### 부트스트랩 
- 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식

### OOB 샘플
- 부트스트랩 샘플에 포함되지않고 남는 샘플
- 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평과할 수 있음(검증 세트의 역할)

In [1]:
import pandas as pd

df = pd.read_csv('http://bit.ly/wine_csv_data')

data = df[['alcohol', 'sugar', 'pH']]
target = df[['class']]


In [2]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target)


In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=1)
print(scores)

C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

{'fit_time': array([0.25684977, 0.28101087, 0.27794075, 0.25895691, 0.2872901 ]), 'score_time': array([0.05479431, 0.0428431 , 0.02934623, 0.05572414, 0.05538988]), 'test_score': array([0.89230769, 0.89230769, 0.88090349, 0.89425051, 0.89117043]), 'train_score': array([0.99794714, 0.99846035, 0.99820421, 0.99820421, 0.99820421])}


In [6]:
rf.fit(train_input, train_target)
rf.feature_importances_

C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


array([0.23740483, 0.49068621, 0.27190896])

In [10]:
rf = RandomForestClassifier(oob_score=True, n_jobs=1)
rf.fit(train_input, train_target)
rf.oob_score_

C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8930623973727422

## 엑스트라트리
- 랜덤 포레스트와 비슷하게 동작하지만 차이점으로는 부트스트랩 샘플 사용하지않음(결정 트리를 만들 때 전체 훈련 세트를 사용, 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위 분할)
- 하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있음
- 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 함
- 엑스트라 트리의 장점으로는 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도 가능

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=1)

scores = cross_validate(et, train_input, train_target, return_train_score=True)
print(scores)

C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

{'fit_time': array([0.31303978, 0.20936966, 0.21258593, 0.21374083, 0.24576831]), 'score_time': array([0.020962  , 0.02193975, 0.02146006, 0.02173638, 0.02885175]), 'test_score': array([0.88820513, 0.88410256, 0.87679671, 0.88603696, 0.89425051]), 'train_score': array([0.99794714, 0.99846035, 0.99820421, 0.99820421, 0.99820421])}


## 그레이디언트 부스팅 
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 방법
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음
- 경사 하강법을 사용하여 트리를 앙상블에 추가
- 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느림

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=500)
scores = cross_validate(gb, train_input, train_target, return_train_score=True)
print(scores)

C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array w

{'fit_time': array([1.53917217, 1.37578368, 1.34792328, 1.33550882, 1.22899318]), 'score_time': array([0.00861835, 0.01292109, 0.00850773, 0.00853658, 0.00894213]), 'test_score': array([0.87384615, 0.86769231, 0.87063655, 0.88398357, 0.87268994]), 'train_score': array([0.92147806, 0.91737234, 0.92047204, 0.91944587, 0.923294  ])}


## 히스토그램 기반 그레이디언트 부스팅
- 입력 특성을 256개의 구간으로 나누기 때문에 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음


In [18]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier()
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(scores)

C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

{'fit_time': array([0.30742431, 0.19708228, 0.18773317, 0.19199944, 0.20688796]), 'score_time': array([0.00849962, 0.00843048, 0.00826573, 0.00719404, 0.01306772]), 'test_score': array([0.87692308, 0.87384615, 0.8798768 , 0.8798768 , 0.87679671]), 'train_score': array([0.9327688 , 0.92789325, 0.93894305, 0.92662904, 0.93381221])}


In [19]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
scores = permutation_importance(hgb, train_input, train_target)
print(scores.importances_mean)

C:\Users\1-03\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.09166667 0.2433908  0.08316913]
